# <span style="color:#134f5c">Advance Transformations

## Extract:

- Import Files:

In [1]:
import pandas as pd

# Read superhero_info.csv
superhero_info_df = pd.read_csv('superhero_info - superhero_info (1).csv')

# Read superhero_powers.csv
superhero_powers_df = pd.read_csv('superhero_powers - superhero_powers.csv')

In [2]:
superhero_info_df.head()

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


In [3]:
superhero_powers_df.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


## Transform:

In [4]:
# Split the 'Hero|Publisher' column into separate columns
superhero_info_df[['Hero', 'Publisher']] = superhero_info_df['Hero|Publisher'].str.split('|', expand=True)

# Separate the measurements column into 2 columns
superhero_info_df[['Height', 'Weight']] = superhero_info_df['Measurements'].str.extract(r"'Height': '([\d.]+) cm', 'Weight': '([\d.]+) kg'")

In [14]:
superhero_info_df.head()

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics,...,0,0,0,0,0,0,0,0,0,0
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics,...,0,0,0,0,0,0,0,0,0,0
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}",Abin Sur,DC Comics,...,0,0,0,0,0,0,0,0,0,0
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",Abomination,Marvel Comics,...,0,0,0,0,0,0,0,0,0,0
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}",Absorbing Man,Marvel Comics,...,0,0,0,0,0,0,0,0,0,0


- Remove:

In [6]:
# Remove 'cm' and 'kg' from Height and Weight columns
superhero_info_df['Height'] = superhero_info_df['Height'].str.replace(' cm', '').astype(float)
superhero_info_df['Weight'] = superhero_info_df['Weight'].str.replace(' kg', '').astype(float)

- OHE the necessary data:

In [7]:
# Explode the 'Powers' column
superhero_powers_df['Powers'] = superhero_powers_df['Powers'].str.split(',')
exploded_powers_df = superhero_powers_df.explode('Powers')

In [8]:
# Get a list of all unique powers
unique_powers = exploded_powers_df['Powers'].dropna().unique()

In [9]:
# Create new columns for each power and encode them
for power in unique_powers:
    superhero_info_df[power] = superhero_info_df['Hero'].isin(exploded_powers_df[exploded_powers_df['Powers'] == power]['hero_names']).astype(int)

C:\Users\casta\AppData\Local\Temp\ipykernel_41856\2572499731.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  superhero_info_df[power] = superhero_info_df['Hero'].isin(exploded_powers_df[exploded_powers_df['Powers'] == power]['hero_names']).astype(int)
C:\Users\casta\AppData\Local\Temp\ipykernel_41856\2572499731.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  superhero_info_df[power] = superhero_info_df['Hero'].isin(exploded_powers_df[exploded_powers_df['Powers'] == power]['hero_names']).astype(int)
C:\Users\casta\AppData

## Load

- merge The DataFrames:

In [11]:
# merge df
merged_df = superhero_info_df.merge(exploded_powers_df, left_on='Hero', right_on='hero_names', how='left')

# Answer the Questions:

In [12]:
# Answer the questions
most_used_height = merged_df.groupby('Publisher')['Height'].apply(lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else None)

# most powerful hero:
most_powerful = merged_df.iloc[:, 11:].sum(axis=1).idxmax()
most_powerful_hero = merged_df.iloc[0]['hero_names']

print("\n Most used height of heroes for each publisher:")
print(most_used_height)
print(f"\n The hero with the most powerful superpower is: [{most_powerful}]\"{most_powerful_hero}\"")


 Most used height of heroes for each publisher:
Publisher
DC Comics            188.0
Dark Horse Comics    183.0
George Lucas         198.0
Image Comics         211.0
Marvel Comics        188.0
Shueisha             168.0
Star Trek            185.0
Team Epic TV         185.0
Unknown              178.0
Name: Height, dtype: float64

 The hero with the most powerful superpower is: [3271]"A-Bomb"


C:\Users\casta\AppData\Local\Temp\ipykernel_41856\1148644856.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  most_powerful = merged_df.iloc[:, 11:].sum(axis=1).idxmax()
